Generate some random data using mllib

In [21]:
import org.apache.spark.SparkContext
import org.apache.spark.mllib.random.RandomRDDs._


// Generate a random double RDD that contains 1 million i.i.d. values drawn from the
// standard normal distribution `N(0, 1)`, evenly distributed in 10 partitions.
val u = normalRDD(sc, 80000000L, 10)
// Apply a transform to get a random double RDD following `N(1, 4)`.
val v = u.map(x => 1.0 + 100000000.0 * x)

In [22]:
val sum = u.take(1000000).reduce((x,y) => x+y)
sum

Double = -479.4090689836632

Create a table using the DataStax driver

In [23]:
import com.datastax.spark.connector._
import com.datastax.spark.connector.cql.CassandraConnector
import org.apache.spark.SparkConf
 
 
 val conf = new SparkConf(true)
 val connector = CassandraConnector(conf)
 connector.withSessionDo(session => {
    session.execute(s"create keyspace if not exists test with replication = { 'class':'SimpleStrategy',"+
          " 'replication_factor':1}")
    session.execute(s"create table if not exists test.test " +
        "(id int, value int, PRIMARY KEY((id)))")
})

com.datastax.driver.core.ResultSet = ResultSet[ exhausted: true, Columns[]]

Insert the random data to the cassandra table

In [24]:
case class TestTableStructure(id: Double, value: Double)

val data = v.map(x => TestTableStructure(x, 1))

data.saveToCassandra("test","test")

In [25]:
val testData = sc.cassandraTable("test","test")
val ids = testData.map(x => x.getDouble("id"))
ids.count()

Do a Kernel Density Test

In [26]:
import org.apache.spark.mllib.stat.KernelDensity
import org.apache.spark.rdd.RDD


// Construct the density estimator with the sample data and a standard deviation for the Gaussian
// kernels
val kd = new KernelDensity().setSample(ids).setBandwidth(.01)

// Find density estimates for the given values
val densities = kd.estimate(Array(1.0,10, 100, 1000,10000, 100000, 1000000, 10000000))

In [27]:
densities

Array[Double] = Array(5.111711219173731E-7, 0.0, 0.0, 0.0, 5.111711219173731E-7, 0.0, 5.111711219173731E-7, 0.0)